In [34]:
#Florian ANDRE
#3B DIA
#BIn312 Data Mining

import os, sys

import requests
from bs4 import BeautifulSoup
import pandas as pd

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
import re



In [35]:
def clean_filename(url):
    return re.sub(r'[\/:*?"<>|]', '_', url)

In [36]:
with open("drones.txt", "r") as f:
    url_list = f.read().splitlines()


for url in url_list:
    response = requests.get(url)
    if response.status_code == 200:
        html_content = response.text  
    else:
        print(f"Failed to retrieve {url}")
    html_content  
    soup = BeautifulSoup(html_content, 'html.parser')
    paragraphs = soup.find_all('p')
    article_text = ' '.join([para.get_text() for para in paragraphs])

    with open(f"scraped_article_{clean_filename(url)}.txt", "w") as f:
        f.write(article_text)

    nltk.download('punkt')
    nltk.download('stopwords')

    with open(f"scraped_article_{clean_filename(url)}.txt", "r") as f:
        article_text = f.read()

    sentences = sent_tokenize(article_text)

    total_sentences = len(sentences)
    num_sentences = int(max(1, int(total_sentences * 0.3))) #le nombre de phrases du résumé est égal à 30% du nombre de phrases de l'article

    #Nettoyage des phrases
    stop_words = set(stopwords.words('english'))
    cleaned_sentences_nltk = []

    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        words = [word for word in words if word not in stop_words and word not in string.punctuation]
        cleaned_sentences_nltk.append(' '.join(words))

    print(cleaned_sentences_nltk)
    cvect = CountVectorizer(ngram_range=(1,1), token_pattern='(?u)\\b\\w+\\b')
    counts = cvect.fit_transform(cleaned_sentences_nltk)
    normalized_counts = normalize(counts, norm='l1', axis=1)

    tfidf = TfidfVectorizer(ngram_range=(1,1))
    tfs = tfidf.fit_transform(cleaned_sentences_nltk)

    sentence_scores = np.sum(tfs.toarray(), axis=1)

    num_sentences = 10
    top_sentence_indices = np.argsort(sentence_scores)[-num_sentences:]

    # Sélection des phrases les plus importantes
    summary = [sentences[i] for i in sorted(top_sentence_indices)]

    
    vectorizer = CountVectorizer().fit_transform(summary)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)

    threshold = 0.7
    redundant_sentences = set()

    for i in range(len(cosine_matrix)):
        for j in range(i + 1, len(cosine_matrix)):
            if cosine_matrix[i][j] > threshold:
                redundant_sentences.add(j)


    filtered_summary = [sentence for idx, sentence in enumerate(summary) if idx not in redundant_sentences]
    
    with open(f"summary_{clean_filename(url)}.txt", "w") as f:
        for sentence in filtered_summary:
            f.write(sentence + "\n")
        f.close()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['profile sections tv featured nbc follow nbc news new alerts time hype around drones may finally starting deliver', 'drone deliveries first touted amazon decade ago slowly taking parts u.s. thursday matternet drone delivery startup launched service silicon valley', '“ years advancing drone delivery healthcare ’ excited launch first home delivery operation heart silicon valley ” said andreas raptopoulos founder ceo matternet blog post', '“ ’ entering next phase scale drone delivery — building new layer ultra-efficient logistics infrastructure become essential utility every household. ” announcement adds signs growth drone delivery', 'fort worth texas recently became first major city united states offer commercial drone deliveries ’ used deliver groceries walmart', 'college station texas amazon ’ drone delivery service become common enough residents see service noisy nuisance', 'recent faa approval company seems set expand drone delivery operations across city beyond', 'experts say many

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['d.c. soon eyes sky', 'metropolitan police department start using drones limited situations try protect residents violent crime', 'comes mayor muriel bowser adds hundreds closed-circuit tv cameras license plate readers', '📺 watch news4 stream nbc4 newscasts free right right', 'taking base near frederick douglass memorial bridge falcon 1 ready action', 'mpd unveiled 6 million chopper press event monday', 'pilot called helicopter “ game changer ” mapping technology infrared cameras help car chases', 'mpd ’ air fleet includes five drones', 'chief police pamela smith said drones used patrol operations used situations including barricades crowd management missing persons cases major crash reconstruction', '“ using artificial intelligence using facial recognition drones yet ” said', 'bowser said d.c. ’ newly approved budget includes funding 200 new cc tv cameras nearly 50 license-plate readers added next year', 'past year d.c. helicopter assisted 76 arrests including carjackings shootings h

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


...

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['print north korea friday accused south korea sending drones drop anti-regime leaflets pyongyang accusation seoul immediately denied', "according state-run korean central news agency north korea 's foreign ministry said south korean drones entered pyongyang night three occasions since october 3 scattering `` numerous '' leaflets", "ministry called alleged incursion `` grave political military provocation '' requires retaliation warning situation could lead armed conflict even war", 'south korea ’ military denied involvement stating send drones would investigate whether private groups may launched leaflets reported south ’ yonhap news agency', 'recent months north korea launched thousands trash-filled balloons south korea retaliation activists float anti-pyongyang leaflets northward across border', 'south korean activists recently employed advanced types balloons send leaflets north korea known reports using drones', "leaflets often criticize north korea 's human rights abuses mock lea

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


["print yemen 's houthi rebels claimed shooting another u.s. military 's mq-9 reaper drones saturday airing footage parts corresponded known pieces unmanned aircraft", 'houthis said shot predator surface-to-air missile part renewed series assaults week rebels relative lull pressure campaign israel-hamas war gaza strip', 'u.s. air force lt. col. bryon j. mcgarry defense department spokesperson acknowledged associated press saturday “ u.s. air force mq-9 drone crashed yemen. ” said investigation underway without elaborating', "houthis described downing happening thursday stronghold country 's saada province", "footage released houthis included described missile launch targeting drone man off-camera reciting houthi 's slogan hit `` god greatest death america death israel curse jews victory islam ''", 'footage included several close-ups parts drone included logo general atomics manufactures drone serial numbers corresponding known parts made company', "since houthis seized country ’ north 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
